In [1]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken chromadb rank_bm25 faiss-cpu
%pip install ingredient_parser_nlp

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
google-cloud-pubsublite 1.8.3 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.4.0 which is incompatible.
jupyterlab 4.0.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.0 requires jupyterlab<5.0.0a0,>=4.0.6, but you have jupyterlab 4.0.5 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.0.1 requires kubernetes<27,>=8.0.0, but you have kubernetes 28.1.0 which 

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.345
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [13]:
from ingredient_parser import parse_multiple_ingredients
import pandas as pd

file_path = "/kaggle/input/final-recipes/final_recipes.xlsx"
# Read the CSV file and handle the error by skipping the problematic row
# Read the Excel file into a pandas DataFrame
try:
    df = pd.read_excel(file_path)
    # Now 'df' contains the data from the Excel file
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

columns_to_remove= ["Unnamed: 0"]
df = df.drop(columns=columns_to_remove, axis=1)
df

,Unnamed: 0.1,Title,Ingredients,Instructions,Cleaned_Ingredients,final_ing
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...","['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","['whole chicken', 'kosher salt', 'acorn squash..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,"['2 large egg whites', '1 pound new potatoes (...","['egg whites', 'new potatoes', 'kosher salt', ..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,"['1 cup evaporated milk', '1 cup whole milk', ...","['evaporated milk', 'whole milk', 'garlic powd..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,"['1 (¾- to 1-pound) round Italian loaf, cut in...","['round italian', 'olive oil', 'sweet italian ..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","['dark brown sugar', 'hot water', 'bourbon', '..."
...,...,...,...,...,...,...
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,"['1 cup all-purpose flour', '2/3 cup unsweeten...","['all-purpose flour', 'unsweetened cocoa powde..."
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,"['1 preserved lemon', '1 1/2 pound butternut s...","['preserved lemon', 'butternut squash', 'olive..."
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",['Leftover katsuo bushi (dried bonito flakes) ...,"['sake', 'sugar', 'soy sauce', 'sesame seeds',..."
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,"['unsalted butter', 'baby spinach', 'feta', 'n..."


In [23]:
import os
import urllib.request
import tarfile
import json
import pandas as pd
import numpy as np
from tqdm import tqdm


#libraries for text preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

#libraries for keyword extraction with tf-idf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix

#libraries for reading and writing files
import pickle

#libraries for BM25
!pip install rank_bm25
from rank_bm25 import BM25Okapi

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
import nltk
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /kaggle/working/...
Archive:  /kaggle/working/corpora/wordnet.zip
   creating: /kaggle/working/corpora/wordnet/
  inflating: /kaggle/working/corpora/wordnet/lexnames  
  inflating: /kaggle/working/corpora/wordnet/data.verb  
  inflating: /kaggle/working/corpora/wordnet/index.adv  
  inflating: /kaggle/working/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/corpora/wordnet/index.verb  
  inflating: /kaggle/working/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/corpora/wordnet/data.adj  
  inflating: /kaggle/working/corpora/wordnet/index.adj  
  inflating: /kaggle/working/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/corpora/wordnet/noun.exc  
  inflating: /kaggle/working/corpora/wordnet/verb.exc  
  inflating: /kaggle/working/corpora/wordnet/README  
  inflating: /kaggle/working/corpora/wordnet/index.sense  
  inflating: /kaggle/working/corpora/wordnet/data.

In [25]:
def preprocess(text):
    #define stopwords
    stop_words = set(stopwords.words("english"))
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)
    #Convert to lowercase
    text = text.lower()
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    ##Convert to list from string
    text = text.split()
    ##Stemming
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  stop_words]
    text = " ".join(text)

    return text

In [26]:
df['processed_ingredients'] = df['final_ing'].apply(lambda x:(preprocess(x)))

In [27]:
pt = df['processed_ingredients'].to_list()


In [75]:
print(len(pt))

13501


In [76]:
pt2 = pt[:3000] 

In [77]:
print(len(pt2))

3000


In [28]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

tagged_documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(pt)]

doc2vec_model = Doc2Vec(tagged_documents, dm=0, vector_size=100, window=2, min_count=1, workers=4)

In [45]:
pip install faiss-cpu


Note: you may need to restart the kernel to use updated packages.


# Hybrid Search

## BM25 Retriever - Sparse retriever

In [41]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

from gensim.models import Word2Vec

In [30]:
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(pt)
bm25_retriever.k = 2

In [42]:
bm25_retriever.get_relevant_documents("Chocolate")

[Document(page_content='fresh poblano chile whole tomato including juice garlic fresh cilantro sugar salt olive oil heavy cream dried epazote soft mild goat cheese salt spinach olive oil raisin salt olive oil corn tortilla black bean garnish toasted pine nut shallow flameproof casserole dish'),
 Document(page_content='romaine lettuce heart treviso radicchio olive oil kosher salt freshly ground pepper mizuna baby mustard green cr fra che avocado fresh lemon juice flaky sea salt')]

In [43]:
bm25_retriever.get_relevant_documents("a green fruit")

[Document(page_content='kiwi fruit'),
 Document(page_content='heavy whipping cream confectioner sugar fruit compote seasonal fruit granulated sugar elderflower ginger syrup')]

## Embeddings - Dense retrievers FAISS

In [86]:
from langchain.embeddings import SentenceTransformerEmbeddings 

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [87]:
faiss_vectorstore = FAISS.from_texts(pt, embeddings)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

Batches:   0%|          | 0/422 [00:00<?, ?it/s]

In [88]:
faiss_retriever.get_relevant_documents("A green fruit")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(page_content='green pear red apple'),
 Document(page_content='kiwi fruit')]

## Ensemble Retriever

In [89]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever],
                                       weights=[0.5, 0.5])

In [90]:
docs = ensemble_retriever.get_relevant_documents("A green fruit")
docs

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(page_content='kiwi fruit'),
 Document(page_content='green pear red apple'),
 Document(page_content='heavy whipping cream confectioner sugar fruit compote seasonal fruit granulated sugar elderflower ginger syrup')]

In [91]:
docs = ensemble_retriever.get_relevant_documents("Butter, Chocolate")
docs

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(page_content='fresh poblano chile whole tomato including juice garlic fresh cilantro sugar salt olive oil heavy cream dried epazote soft mild goat cheese salt spinach olive oil raisin salt olive oil corn tortilla black bean garnish toasted pine nut shallow flameproof casserole dish'),
 Document(page_content='chocolate'),
 Document(page_content='unsalted butter'),
 Document(page_content='romaine lettuce heart treviso radicchio olive oil kosher salt freshly ground pepper mizuna baby mustard green cr fra che avocado fresh lemon juice flaky sea salt')]